<a href="https://colab.research.google.com/github/tayfununal/hamiltonian_ex/blob/main/solve_ivp_RK45.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Ode solver için gerekli kütüphaneler
import matplotlib.pyplot as plt
import numpy as np
from scipy.integrate import solve_ivp

# Deep learning için gerekli kütüphaneler 
import tensorflow as tf
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Dense

plt.style.use('seaborn-poster')

%matplotlib inline
"""
def F(t,s):       # Diff eq F(t,s) = 0
  return [np.cos(t), -np.sin(t)]
"""
#tf.random.set_seed(39)
#np.random.seed(39)

initial_state = [1., 0.]                  # t0 anındaki başlangıç değerleri
t_starting_and_last = [0, 2*np.pi]        # Fonksiyonun tanımlı olacağı alanın başlangıç ve bitiş noktasını tanımlıyoruz.
t_eval = np.arange(0, 2*np.pi, 0.5)      # t_starting_and_last ile belirlediğimiz aralıkta hesaplanacak t değerlerini bu şekilde girebiliriz, hiçbirşey girilmez ise default olarak kendisi parçalıyor.

def F(t,y):       # Diff eq F(t,s) = 0
  return [y[1], -y[0]]

sol = solve_ivp(fun= F, t_span= t_starting_and_last, y0= initial_state, t_eval= t_eval, dense_output=True)

q = sol.y[0]
p = sol.y[1]
t = sol.t

In [ ]:
plt.figure(figsize = (8, 8))

plt.subplot(221)
plt.plot(t, q)
plt.xlabel('t')
plt.ylabel('q = cos(t)')

plt.subplot(222)
plt.plot(t, q - np.cos(t))
plt.xlabel('t')
plt.ylabel('q - cos(t)')

plt.subplot(223)
plt.plot(t, p)
plt.xlabel('t')
plt.ylabel('p = sin(t)')

plt.subplot(224)
plt.plot(t, p + np.sin(t))
plt.xlabel('t')
plt.ylabel('p + sin(t)')

plt.tight_layout()
plt.show()

In [ ]:
def grand_truth_plot(h = 0.1):
  t = np.arange(0,2*np.pi,0.3)
  q = np.cos(t)
  p = np.sin(t)
  plt.scatter(q,p)

def pred_plot(q, p):
  plt.plot(q[:, ], p[:,], c = "red")
  plt.xlabel('q')
  plt.ylabel('p')

In [ ]:
plt.figure(figsize = (4, 4))

pred_plot(q=q, p=p)
grand_truth_plot()

plt.show()

In [ ]:
target = np.concatenate((q.reshape(-1,1), p.reshape(-1,1)) , axis=1)

In [ ]:
inp = Input(shape=(1,))

x = Dense(64, activation="tanh")(inp)
x = Dense(64, activation="tanh")(x)

out = Dense(2)(x)

model = Model(inputs=inp, outputs=out)
model.summary()

In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate=0.001)

model.compile(optimizer=opt , loss="mse")
model.fit(x=t, y=target , epochs=1000, batch_size=16)

In [ ]:
# prediction q and p Using NN and with test_t 
test_t = np.arange(0, 2*np.pi, 0.001)
prediction = model(test_t).numpy()

q = prediction[:,0]
p = prediction[:,1]

plt.figure(figsize = (4, 4))

# NN with mse solition 
pred_plot(q, p )

# Grand truth solution plot
grand_truth_plot()

plt.show()

In [ ]:
# NN model with Hamiltonian preserved loss
inp = Input(shape=(2,))

x = Dense(64, activation="tanh")(inp)
x = Dense(64, activation="tanh")(x)

out = Dense(2)(x)

model2 = Model(inputs=inp, outputs=out)

In [ ]:
def H(z):
  return tf.reduce_sum((z**2)/2,axis=1)

def custom_loss(y_true, y_pred):
  return (1/32) *(y_true - H(y_pred))**2

In [ ]:
# Hamiltonian preserved for initial point and we extend this for all test_t
h_0 = np.array([0.5 for i in range(test_t.shape[0])])

In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate=0.001)

model2.compile(optimizer=opt, loss=custom_loss)
model2.fit(x = prediction, y = h_0, epochs=100, batch_size=32)

In [ ]:
test_t = np.arange(0, 2*np.pi, 0.0001)

prediction_with_hamiltonian_preserved = model2(model(test_t))

q = prediction_with_hamiltonian_preserved[:,0]
p = prediction_with_hamiltonian_preserved[:,1]

plt.figure(figsize = (4, 4))
pred_plot(q=q, p=p)

grand_truth_plot()
plt.show()